In [1]:
from tensorflow.keras.models import load_model
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

Model load and prediction

In [2]:
model = load_model('LSTM_euler_temporal_dynamics')

In [3]:
sample_input_sequence = np.array([[-9.28317111e-01, -9.26705018e-01, -1.33821092e+00,
         1.59151996e-01, -9.80534562e-01,  1.13180968e+00,
        -8.14734264e-01, -1.29092260e-02, -1.01997518e+00,
         2.10817179e+00,  2.12091685e-02, -1.51470508e-03,
        -4.27905404e-02,  1.15207013e+00,  1.26888526e+00,
        -6.85882976e-01, -1.34530801e-01, -2.87846841e-02,
        -8.30967007e-01,  2.60652480e+00, -2.04741732e-01,
        -1.45619779e-03]]) # 1 rep of certain exercise

In [13]:
sample_noisy_sequence = sample_input_sequence.copy()

# Adjustments
for i, value in enumerate(sample_noisy_sequence[0]):
    if i in {2, 4, 5, 6}:  # angle indices
        # Shift by a random angle and wrap around using np.clip to ensure the value stays within -pi to pi
        new_value = value + np.random.uniform(0.5, -1.0)
        sample_noisy_sequence[0][i] = np.clip(new_value, -np.pi, np.pi)
    else:
        # Apply random noise
        noise = np.random.normal(0, 0.1)
        sample_noisy_sequence[0][i] = value + noise # Adjust the scale of noise as needed

In [14]:
sample_sequence = np.expand_dims(sample_input_sequence, axis=1)

In [15]:
sample_noisy_sequence = np.expand_dims(sample_noisy_sequence, axis=1)

In [16]:
predictions = model.predict(sample_sequence)

1/1 [==============================] - 0s 9ms/step


In [17]:
categorical_features = ['armraise', 'tricepkickback', 'overheadpress', 'bicepcurl',
                        'hammercurl', 'burpee', 'vup', 'crunch', 'pushup', 'birddog'] 
categories_array = np.array(categorical_features).reshape(-1, 1)
encoder = OneHotEncoder(sparse_output=False)
target_encoded = encoder.fit_transform(categories_array)

In [18]:
predicted_indices = np.argmax(predictions, axis=1)
print(predicted_indices)
predicted_one_hot = np.zeros((predicted_indices.size, predictions.shape[1]))
predicted_one_hot[np.arange(predicted_indices.size), predicted_indices] = 1

predicted_labels = encoder.inverse_transform(predicted_one_hot)

[2]


In [19]:
predicted_labels

array([['birddog']], dtype='<U14')

Task -3 : Noisy Data Handling (basic)

In [34]:
def classify_with_threshold(predictions, encoder, threshold=0.7):
    # Identify max probabilities and their corresponding indices
    max_probs = np.max(predictions, axis=1)
    predicted_labels = np.argmax(predictions, axis=1)
    print("Max probabilities:", max_probs)
    print("Predicted labels before thresholding:", predicted_labels)

    # Mask for where max probabilities are below the threshold
    other_mask = max_probs < threshold
    print("Indices below threshold:", np.where(other_mask)[0])  # Print indices below threshold for debugging

    # Create one-hot encoding for predicted labels
    num_categories = len(encoder.categories_[0])
    predicted_one_hot = np.zeros((predicted_labels.size, num_categories))  # No extra column for 'other'
    predicted_one_hot[np.arange(predicted_labels.size), predicted_labels] = 1

    # Decode labels back from one-hot encoding
    decoded_labels = encoder.inverse_transform(predicted_one_hot)

    # Assign 'other' to those indices where max_probs < threshold
    decoded_labels[other_mask] = 'other'
    return decoded_labels

In [35]:
predictions = model.predict(sample_noisy_sequence)
print("Raw model predictions:", predictions)
final_predictions = classify_with_threshold(predictions, encoder, threshold=0.7)
print("Final predictions with threshold:", final_predictions)

1/1 [==============================] - 0s 10ms/step
Raw model predictions: [[7.9563833e-06 8.8968220e-05 5.0660479e-01 4.2282179e-01 5.5222819e-07
  7.0224449e-02 2.1924463e-04 3.2012067e-05 2.8831846e-07 9.0518171e-08]]
Max probabilities: [0.5066048]
Predicted labels before thresholding: [2]
Indices below threshold: [0]
Final predictions with threshold: [['other']]
